In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linalg

**Matrices Python (Repaso):**

In [ ]:
prueba = np.asmatrix(np.array([[8, 14, -6], [12,7,4], [-11,3,21]]))

In [ ]:
pd.DataFrame(prueba)

In [ ]:
prueba[:,1] #Columna
prueba[1,:] #Fila

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(prueba)

In [ ]:
eig_vals[1]*eig_vecs[:,1], prueba@eig_vecs[:,1]

**Funciones SSH:**

In [ ]:
def H_t1_t2(N, delta, PBC = False):
    Matriz = np.zeros((N,N))

    a = 0 #diagonal
    b = 1 + delta #la que aparece despues - en este caso sería -t2
    c = 1 - delta #la que aparece primero - en este caso sería t1

    for i in range(N):
        Matriz[i,i] = a
    for i in range(N-1):
        if (i%2 == 0):
            Matriz[i+1,i] = c
            Matriz[i,i+1] = c
        else:
            Matriz[i+1,i] = b
            Matriz[i,i+1] = b
            
    if PBC:
        Matriz[0,N-1] = b #PBC
        Matriz[N-1,0] = b #PBC
            
    return Matriz  

def Diagonalizar(Matriz):
    
    eig_vals, eig_vecs = np.linalg.eig(Matriz)
    eig_vals_sorted = np.sort(eig_vals)
    eig_vecs_sorted = eig_vecs[:, eig_vals.argsort()]
    
    N = Matriz.shape[0]
    D = np.identity(N)*eig_vals_sorted
    Q = np.asmatrix(eig_vecs_sorted)

    return D, Q, np.asmatrix(Matriz)

def Estado_Base(D,U):
    i = 0
    while D[i,i] <= 0:
        i = i +1
    indice = i #Ese es la energía del estado base:    
    Vec_Est_Base = U[:,indice]
    lambda_base = D[indice,indice]
    
    return Vec_Est_Base/np.linalg.norm(Vec_Est_Base), lambda_base #Retornemoslo normalizado

**Funciones Algoritmo de Lanczos:**

In [ ]:
def revisar_ortogonalidad(Lanczos_Matrix):
    N = Lanczos_Matrix.shape[1]
    
    for i in range(N):
        for j in range(N):
            Mensaje = "< u_" + str(i) + "| u_" + str(j) + "> = "
            valor = float(np.transpose(Lanczos_Matrix[:,i])@ Lanczos_Matrix[:,j])
            Mensaje = Mensaje + str(valor)
            print(Mensaje)
            
def revisar_tridiagonal(Lanczos_Matrix, a_coef, b_coef, H):
    N = Lanczos_Matrix.shape[1]
    
    for n in range(1,N - 1):
        Mensaje = "|| H|u_" + str(n) + "> - (a_" + str(n) + "|u_" + str(n) +"> + b_" +str(n+1) + "|u_" + str(n+1) + " + b_" +str(n) + "|u_" + str(n-1) + ">) || = "
        Mensaje = Mensaje + str(float(np.linalg.norm(H@Lanczos_Matrix[:,n] - ( a_coef[n]*Lanczos_Matrix[:,n] + b_coef[n+1]*Lanczos_Matrix[:,n+1] + b_coef[n]*Lanczos_Matrix[:,n-1]))))
        print(Mensaje)
            
def Refinar_base(Lanczos_Matrix, a_coef, b_coef, H):
    N = Lanczos_Matrix.shape[1]
    indice_error_tridiagonal = N

    for n in range(1,N - 1):
        tridiagonalidad_propiedad_n = float(np.linalg.norm(H@Lanczos_Matrix[:,n] - ( a_coef[n]*Lanczos_Matrix[:,n] + b_coef[n+1]*Lanczos_Matrix[:,n+1] + b_coef[n]*Lanczos_Matrix[:,n-1])))
        if not (np.isclose(tridiagonalidad_propiedad_n, 0)): 
            indice_error_tridiagonal = n
            break

    indice_error_ortogonalidad = N
    for i in range(N):
        indices_error_ortogonalidad = []
        for j in range(i):
            ortogonalidad_ij = float(np.transpose( Lanczos_Matrix[:,i])@ Lanczos_Matrix[:,j])
            if not (np.isclose(ortogonalidad_ij, 0)):
                indices_error_ortogonalidad.append(i)
        if (len(indices_error_ortogonalidad) == 0): continue
        if (np.min(indices_error_ortogonalidad) <= indice_error_ortogonalidad): indice_error_ortogonalidad = np.min(indices_error_ortogonalidad)

    limite = min(indice_error_ortogonalidad, indice_error_tridiagonal)     
    
    return Lanczos_Matrix[:,:limite], a_coef[:limite], b_coef[:limite+1]

**Funciones Spread Complexity:**

In [ ]:
def spread_complexity_using_lanczos_basis(tiempos, Lanczos_Basis, H):
    complejidad = np.array([])
    u_0 = Lanczos_Basis[:,0]

    for i in range(len(tiempos)):
        t = tiempos[i]
        u_t = np.dot(linalg.expm((-1j)*H*t), u_0)
        
        if(Lanczos_Basis.shape[0] == Lanczos_Basis.shape[1]): coefs_phi_t = np.linalg.inv(Lanczos_Basis)@u_t
        else: coefs_phi_t = np.linalg.pinv(Lanczos_Basis)@u_t
        
        c_t = 0
        for n in range(len(coefs_phi_t)):
            c_t = c_t + n*np.abs(coefs_phi_t[n])**2
        complejidad = np.append(complejidad, c_t)
        
    return complejidad